# Leitura e Formatação dos Dados de Desmatamento

In [ ]:
import geopandas as gpd
import ast
import requests
import os
import rasterio
import pandas as pd

# Carregar o arquivo GeoPackage
gdf = gpd.read_file(r"G:\Meu Drive\Desmatamento - Alertas [2024-11]\deforestation_alerts_awa.gpkg")
gdf.columns

Index(['is_valid', 'analysis_date', 'area', 'location', 'source_images_date',
       'source_images', 'internal_storage_path', 'geometry'],
      dtype='object')

In [ ]:
# Caminho da pasta principal que contém os seis lugares
caminho_principal = "/content/drive/MyDrive/Desmatamento_Arquivos/"

# Lista para armazenar os dados
dados = []

# Itera sobre cada lugar
for lugar in os.listdir(caminho_principal):
    caminho_lugar = os.path.join(caminho_principal, lugar)

    if os.path.isdir(caminho_lugar):
        # Encontra subdiretórios "PROD"
        for subdir in os.listdir(caminho_lugar):
            if subdir.startswith('PROD'):
                caminho_prod = os.path.join(caminho_lugar, subdir)

                # Acessa a pasta de data e ID
                for data_produto in os.listdir(caminho_prod):
                    caminho_data_produto = os.path.join(caminho_prod, data_produto)

                    # Verifica se é um diretório
                    if os.path.isdir(caminho_data_produto):
                        data, id_produto = data_produto.split('_', 1)  # Extrai data e ID

                        # Percorre cada banda
                        for banda in os.listdir(caminho_data_produto):
                            caminho_banda = os.path.join(caminho_data_produto, banda)

                            # Apenas adiciona diretórios de banda
                            if os.path.isdir(caminho_banda):
                                dados.append({
                                    'Lugar': lugar,
                                    'Produto': subdir,
                                    'Data_Produto': data_produto,
                                    'Banda': banda,
                                    'Caminho_Completo': caminho_banda,
                                    'Data_Imagem': data,
                                    'ID_Produto': id_produto
                                })
# Criação do DataFrame
df = pd.DataFrame(dados)
end_bands = ['_B02_10m.jp2', '_B03_10m.jp2', '_B04_10m.jp2', '_B08_10m.jp2', '_SCL_20m.jp2', '_TCI_10m.jp2']


(87,)

In [2]:
# Função para baixar um arquivo
def download_file(url, save_path):
    # Verificar se o arquivo já existe
    if os.path.exists(save_path):
        print(f"Arquivo já existe: {save_path}")
        return
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Download concluído: {save_path}")
    else:
        print(f"Falha no download: {url}")
        print(f"Nome: {save_path}")
        
def criar_urls_bandas(source_images, bands, resolutions):
    list_source_images = []
    for source_image in source_images:
        source_images_full = []
        if source_image['Band'] == 'B04':
            url = source_image['Url']
            name = source_image['Name']
            band = source_image['Band']
            for j in range(14):
                dict_link = url.replace('B04_10m', bands[j])
                dict_link = dict_link.replace('R10m', resolutions[j])
                
                dict_name = name.replace('B04_10m', bands[j])
                dict_band = band.replace('B04', bands[j].split('_')[0])
                
                dados = {'Name': dict_name, 'Band': dict_band, 'Url': dict_link}
                
                source_images_full.append(dados)
                
        list_source_images.append(source_images_full)
                
                
    return list_source_images    
            

In [3]:
list_bands_sentinel = ['B01_60m', 
                       'B02_10m',
                       'B03_10m',
                       'B04_10m',  
                       'B05_20m', 
                       'B06_20m',
                       'B07_20m',
                       'B08_10m',
                       'B8A_20m',
                       'B09_60m',
                       'B11_20m',
                       'B12_20m',
                       'SCL_20m',
                       'TCI_10m',]

list_bands_resolutions = ['R60m', 
                          'R10m', 
                          'R10m', 
                          'R10m', 
                          'R20m', 
                          'R20m', 
                          'R20m', 
                          'R10m',
                          'R20m',
                          'R60m',
                          'R20m',
                          'R20m',
                          'R20m',
                          'R10m']

In [48]:
import os
list_final_source_images = []
save_directory = 'C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/'
for i in range(0, 50):
    row = gdf.iloc[i]
    name_path = save_directory + row['internal_storage_path'] + '/'
    url_b04_merge = f"https://objectstorage.sa-saopaulo-1.oraclecloud.com/n/grffufe7dd4e/b/carboneye/o/{row['internal_storage_path']}/B04_merged.tiff"
    url_b08_merge = f"https://objectstorage.sa-saopaulo-1.oraclecloud.com/n/grffufe7dd4e/b/carboneye/o/{row['internal_storage_path']}/B08_merged.tiff"
    url_tci_merge = f"https://objectstorage.sa-saopaulo-1.oraclecloud.com/n/grffufe7dd4e/b/carboneye/o/{row['internal_storage_path']}/TCI_merged.tiff"
    url_scl_merge = f"https://objectstorage.sa-saopaulo-1.oraclecloud.com/n/grffufe7dd4e/b/carboneye/o/{row['internal_storage_path']}/SCL_merged.tiff"
    source_images = ast.literal_eval(row['source_images'])
    list_source_images = criar_urls_bandas(source_images, list_bands_sentinel, list_bands_resolutions)
    #list_final_source_images.append(list_source_images)
    for source_images in list_source_images:
        for source_image in source_images:
            url = source_image['Url']
            name = source_image['Name']
            save_path = name_path + name[:-12] + '/'
            os.makedirs(save_path, exist_ok=True)
            download_file(url, save_path + name)
            
    download_file(url_b04_merge, name_path + 'B04_merged.tiff')
    download_file(url_b08_merge, name_path + 'B08_merged.tiff')
    download_file(url_tci_merge, name_path + 'TCI_merged.tiff')
    download_file(url_scl_merge, name_path + 'SCL_merged.tiff')

Arquivo já existe: C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/PROD_5df78447-f1d2-46b6-80ff-e148a457d671/20240103_50754cf8-052f-4eb1-9b71-56e9e413fdb9/T22MCC_20240103T134709/T22MCC_20240103T134709_B01_60m.jp2
Arquivo já existe: C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/PROD_5df78447-f1d2-46b6-80ff-e148a457d671/20240103_50754cf8-052f-4eb1-9b71-56e9e413fdb9/T22MCC_20240103T134709/T22MCC_20240103T134709_B02_10m.jp2
Arquivo já existe: C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/PROD_5df78447-f1d2-46b6-80ff-e148a457d671/20240103_50754cf8-052f-4eb1-9b71-56e9e413fdb9/T22MCC_20240103T134709/T22MCC_20240103T134709_B03_10m.jp2
Arquivo já existe: C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/PROD_5df78447-f1d2-46b6-80ff-e148a457d671/20240103_50754cf8-052f-4eb1-9b71-56e9e413fdb9/T22MCC_20240103T134709/T22MCC_20240103T134709_B04_10m.jp2
Arquivo já existe: C:/Users/alanb/Documents/Desmatamento_Arquivos/awa/PROD_5df78447-f1d2-46b6-80ff-e148a457d671/20240103_50754cf8-052f-4

KeyboardInterrupt: 

In [16]:
INTERNAL_STORAGE_PATH = gdf['internal_storage_path'][0]

In [18]:
gdf_01 = gdf[gdf['internal_storage_path'] == INTERNAL_STORAGE_PATH]

In [27]:
gdf['internal_storage_path'].unique().shape

(77,)

In [10]:
row = gdf.iloc[0]
source_images = row['source_images']
source_images = ast.literal_eval(source_images)

# Diretório onde os arquivos serão salvos
save_directory = 'C:/Users/alanb/Documents'

In [ ]:
# Baixar cada arquivo da lista de links
for link in source_images:
    print(link['Url'])
    file_name = link['Url'].split('/')[-1]
    save_path = f"{save_directory}/{file_name}"
    download_file(link['Url'], save_path)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon

# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Criar uma figura e um eixo
fig, ax = plt.subplots()

# Plotar cada polígono no MultiPolygon
for polygon in multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot')

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import MultiPolygon
from shapely.ops import transform
from pyproj import Proj, Transformer

# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Caminho para a imagem
image_path = r"C:\Users\alanb\Documents\TCI_merged.tiff"

# Ler a imagem usando rasterio
with rasterio.open(image_path) as src:
    image = src.read([1, 2, 3])  # Ler as três primeiras bandas (RGB)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    image_crs = src.crs  # Sistema de coordenadas da imagem

# Sistema de coordenadas dos polígonos (supondo que seja WGS84)
polygons_crs = 'EPSG:4326'

# Criar um transformador para converter as coordenadas
transformer = Transformer.from_crs(polygons_crs, image_crs, always_xy=True)

# Função para transformar as coordenadas dos polígonos
def transform_polygon(polygon):
    return transform(transformer.transform, polygon)

# Transformar os polígonos para o sistema de coordenadas da imagem
transformed_multipolygon = MultiPolygon([transform_polygon(polygon) for polygon in multipolygon.geoms])

# Criar uma figura e um eixo
fig, ax = plt.subplots(figsize=(10, 10))

# Plotar a imagem de fundo
ax.imshow(image.transpose(1, 2, 0), extent=extent)

# Plotar cada polígono no MultiPolygon transformado
for polygon in transformed_multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y, color='red')
    print(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot with Background Image')

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import MultiPolygon
from shapely.ops import transform
from pyproj import Proj, Transformer
import numpy as np


# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Caminho para a imagem
image_path = r"C:\Users\alanb\Documents\B04_merged.tiff"

# Ler a imagem usando rasterio
with rasterio.open(image_path) as src:
    image = src.read()  # Ler as três primeiras bandas (RGB)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    image_crs = src.crs  # Sistema de coordenadas da imagem

# Sistema de coordenadas dos polígonos (supondo que seja WGS84)
polygons_crs = 'EPSG:4326'

# Criar um transformador para converter as coordenadas
transformer = Transformer.from_crs(polygons_crs, image_crs, always_xy=True)

# Função para transformar as coordenadas dos polígonos
def transform_polygon(polygon):
    return transform(transformer.transform, polygon)

# Transformar os polígonos para o sistema de coordenadas da imagem
transformed_multipolygon = MultiPolygon([transform_polygon(polygon) for polygon in multipolygon.geoms])

# Criar uma figura e um eixo
fig, ax = plt.subplots(figsize=(10, 10))

# Plotar a imagem de fundo
ax.imshow(np.squeeze(image), extent=extent)

# Plotar cada polígono no MultiPolygon transformado
for polygon in transformed_multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y, color='red')
    print(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot with Background Image')

# Mostrar o gráfico
plt.show()